### 1. Configuração do ambiente e carregamento de dados

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [46]:
# Carregamento da tabela principal
caminho = '../data/raw/dados_brutos.csv'
df_psico = pd.read_csv(caminho)

In [47]:
# Carregamento de dicionários
dic_sid = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_cid_10.csv')
dic_municipio = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_municipio.csv')
dic_geral = pd.read_csv('../data/dicionarios/br_ms_sia_dicionario.csv')

### 2. Entendimento e limpeza de dados

#### 2.1. Limpeza de dados desnecessários

Como a nossa análise vai focar na região, CID, raça/cor, análise temporal e idade. Podemos remover colunas desnecessárias e também nulas

In [48]:
df_psico.columns

Index(['ano', 'mes', 'sigla_uf', 'id_municipio', 'id_estabelecimento_cnes',
       'id_estabelecimento_cnes_familia', 'id_procedimento_ambulatorial',
       'id_servico_especializado', 'id_classificacao_servico',
       'data_inicio_atendimento', 'data_termino_atendimento',
       'permanencia_atendimento', 'motivo_saida_permanencia',
       'data_motivo_saida_permanencia', 'ano_processamento',
       'mes_processamento', 'ano_atendimento', 'mes_atendimento',
       'data_nascimento_paciente', 'id_municipio_residencia_paciente',
       'origem_paciente', 'nacionalidade_paciente', 'tipo_idade',
       'idade_paciente', 'sexo_paciente', 'raca_cor_paciente',
       'etnia_paciente', 'carater_atendimento', 'cid_principal_categoria',
       'cid_principal_subcategoria', 'cid_causas_associadas_categoria',
       'cid_causas_associadas_subcategoria', 'tipo_droga', 'destino_paciente',
       'local_realizacao_atendimento', 'indicador_situacao_rua',
       'indicador_estrategia_familia', 'quant

In [49]:
# Limpeza de nulos
df = df_psico
percentual_nulos = (df.isnull().sum() / len(df)) * 100 # percentual

colunas_com_nulos = percentual_nulos[percentual_nulos > 0].sort_values(ascending=False)

print("Percentual de valores nulos por coluna:")
print(colunas_com_nulos)

Percentual de valores nulos por coluna:
indicador_situacao_rua                100.000000
indicador_estrategia_familia          100.000000
data_motivo_saida_permanencia          98.534204
cid_causas_associadas_categoria        97.656201
cid_causas_associadas_subcategoria     93.852146
data_inicio_atendimento                79.274803
data_termino_atendimento               79.274803
cid_principal_categoria                65.129777
cid_principal_subcategoria             34.870223
dtype: float64


In [50]:
remover = ['indicador_situacao_rua', 'indicador_estrategia_familia', 'data_motivo_saida_permanencia', 'cid_causas_associadas_categoria', 
           'cid_causas_associadas_subcategoria', 'local_realizacao_atendimento', 'quantidade_atendimentos', 'quantidade_pacientes',
           'sigla_uf', 'nacionalidade_paciente', 'origem_paciente', 'nacionalidade_paciente', 'tipo_idade', 'ano_processamento' ,
           'id_estabelecimento_cnes', 'id_estabelecimento_cnes_familia', 'id_classificacao_servico', 'mes_processamento',
           'data_inicio_atendimento', 'data_termino_atendimento', 'permanencia_atendimento', 'motivo_saida_permanencia', 'etnia_paciente',
           'carater_atendimento', 'tipo_droga', 'destino_paciente'
           ]

df_psico = df_psico.drop(columns=remover)

In [51]:
# Remoção de anos menores que 2014 e maiores que 2024 (fora da série temporal da análise):
anos_remover = [2012, 2013, 2025]

df_psico = df_psico[~df_psico['ano'].isin(anos_remover)]

In [52]:
df_psico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3412672 entries, 5 to 3717638
Data columns (total 16 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   ano                                int64 
 1   mes                                int64 
 2   id_municipio                       int64 
 3   id_procedimento_ambulatorial       int64 
 4   id_servico_especializado           int64 
 5   ano_atendimento                    int64 
 6   mes_atendimento                    int64 
 7   data_nascimento_paciente           object
 8   id_municipio_residencia_paciente   int64 
 9   idade_paciente                     int64 
 10  sexo_paciente                      object
 11  raca_cor_paciente                  int64 
 12  cid_principal_categoria            object
 13  cid_principal_subcategoria         object
 14  quantidade_produzida_procedimento  int64 
 15  quantidade_aprovada_procedimento   int64 
dtypes: int64(12), object(4)
memory usage: 442

#### 2.2. Junção com dicionários

In [53]:
# Municípios
dic_nomes = dic_municipio.set_index('id_municipio')['nome']

# adiciona nome dos municípios de atendimento para a tabela inicial
df_psico['nome_municipio'] = df_psico['id_municipio'].map(dic_nomes)

# adiciona nome dos municípios de residencia dos pacientes para a tabela inicial
df_psico['nome_municipio_residencia'] = df_psico['id_municipio_residencia_paciente'].map(dic_nomes)

In [54]:
# remoção de colunas que não serão mais usadas
df_psico = df_psico.drop(columns=['id_municipio', 'id_municipio_residencia_paciente'])

In [55]:
colunas_chave = ['nome_coluna', 'chave']
dic_geral = dic_geral.drop_duplicates(subset=colunas_chave, keep='first')

In [56]:
# adiciona a etnia dos pacientes
raca_cor = dic_geral[dic_geral['nome_coluna'] == 'raca_cor_paciente']
mapa = raca_cor.set_index('chave')['valor']
df_psico['raca_cor_paciente'] = df_psico['raca_cor_paciente'].astype(str).map(mapa)

In [57]:
df_psico.head()

,ano,mes,id_procedimento_ambulatorial,id_servico_especializado,ano_atendimento,mes_atendimento,data_nascimento_paciente,idade_paciente,sexo_paciente,raca_cor_paciente,cid_principal_categoria,cid_principal_subcategoria,quantidade_produzida_procedimento,quantidade_aprovada_procedimento,nome_municipio,nome_municipio_residencia
5,2014,2,301080208,115,2014,2,1986-09-09,27,M,sem informacao,NaN,F102,1,1,São José dos Pinhais,São José dos Pinhais
6,2014,2,301080240,115,2014,2,1967-01-27,46,M,parda,NaN,F102,1,1,Lapa,Lapa
7,2014,2,301080194,115,2014,2,1980-04-09,33,F,parda,NaN,F430,1,1,Piraquara,Piraquara
8,2014,2,301080208,115,2014,2,1973-07-14,39,F,parda,F45,NaN,1,1,Piraquara,Piraquara
9,2014,2,301080194,115,2014,2,1954-10-23,59,F,parda,F32,NaN,1,1,Piraquara,Piraquara


Removemos os dados em que a coluna quantidade_aprovada_procedimento são igual a 0. 

In [58]:
df_psico = df_psico[df_psico['quantidade_aprovada_procedimento'] > 0].copy()

#### 2.3. Análise descritiva e limpeza adicional

Identificação de dados ausentes e nulos

In [59]:
df_psico.shape

(3125441, 16)

In [62]:
pd.options.display.float_format = '{:.2f}'.format

In [63]:
# Análise dos dados quantitativos
df_psico.describe()

,ano,mes,id_procedimento_ambulatorial,id_servico_especializado,ano_atendimento,mes_atendimento,idade_paciente,quantidade_produzida_procedimento,quantidade_aprovada_procedimento
count,3125441.00,3125441.00,3125441.00,3125441.00,3125441.00,3125441.00,3125441.00,3125441.00,3125441.00
mean,2019.32,6.70,301080222.47,115.00,2019.31,6.70,36.14,1.39,1.39
std,3.17,3.33,52.69,0.00,3.17,3.33,16.07,3.20,2.44
min,2014.00,1.00,301080020.00,115.00,2013.00,1.00,0.00,1.00,1.00
25%,2017.00,4.00,301080208.00,115.00,2017.00,4.00,23.00,1.00,1.00
50%,2019.00,7.00,301080216.00,115.00,2019.00,7.00,38.00,1.00,1.00
75%,2022.00,10.00,301080224.00,115.00,2022.00,10.00,49.00,1.00,1.00
max,2024.00,12.00,301080356.00,115.00,2024.00,12.00,97.00,3027.00,902.00


As variáveis quantidade produzidas e aprovadas aparentam estar com um outliner extremo de quantidade máxima de procedimento

In [73]:
df_psico['quantidade_aprovada_procedimento'].value_counts()

quantidade_aprovada_procedimento
1      2749616
2       150755
3        76465
4        51077
5        35397
6        17463
7         9521
8         8430
10        6545
9         4517
12        3079
14        2504
11        2263
13        1191
15         829
18         614
24         546
16         539
48         275
20         257
32         204
21         202
17         184
36         159
30         152
25         143
19         139
23         126
46         126
45         116
26         108
27         107
22         104
28         102
38          79
41          77
34          72
35          71
54          70
49          69
29          56
98          45
40          45
33          39
39          38
31          36
37          34
50          34
42          33
47          32
78          31
43          29
68          29
58          27
64          24
75          23
52          22
56          20
57          19
79          18
51          17
53          16
44          15
124         15
70     

Isso comprova que a variável tem vários outliners que podem distorcer a análise

In [74]:
# Análise de variáveis qualitativas
df_psico.describe(include=['object'])

,data_nascimento_paciente,sexo_paciente,raca_cor_paciente,cid_principal_categoria,cid_principal_subcategoria,nome_municipio,nome_municipio_residencia
count,3125441,3125441,3125441,1114674,2010767,3125441,3125441
unique,24977,2,6,139,476,17,95
top,1967-12-01,M,branca,F99,F102,Curitiba,Curitiba
freq,2040,1791016,1564910,357258,415133,1911284,1799427
